# Smolagents (Hugging Face)

**Objectifs**  
- Découvrir les **CodeAgent** et **ToolCallingAgent** avec **smolagents**.  
- Utiliser un modèle **OpenAI** ou **Gemini** via `OpenAIServerModel` (vous avez `OPENAI_API_KEY` ou `GOOGLE_API_KEY`).  
- Construire 2 outils : `calculator` (arithmétique) et `kb_lookup` (mini base locale).  
- Comparer **CodeAgent** vs **ToolCallingAgent** sur 2–3 tâches.

> Docs utiles : guided tour, outils, modèles (OpenAI/Gemini).  
> **Note** : Définissez `USE_GEMINI=1` dans votre `.env` pour utiliser Gemini au lieu d'OpenAI.


## 0) Installation & Setup

In [ ]:
import os

# Try to load from .env file if available (optional)
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  # dotenv not installed, that's okay

# Determine which backend to use
USE_GEMINI = os.environ.get("USE_GEMINI", "").lower() in ("1", "true", "yes")

if USE_GEMINI:
    assert os.getenv("GOOGLE_API_KEY"), "⚠️ Please set GOOGLE_API_KEY in your environment!"
    print("✅ OK: GOOGLE_API_KEY detected (Gemini mode)")
else:
    assert os.getenv("OPENAI_API_KEY"), "⚠️ Please set OPENAI_API_KEY in your environment!"
    print("✅ OK: OPENAI_API_KEY detected (OpenAI mode)")


## 1) Choisir le modèle (OpenAI ou Gemini)


In [ ]:
from smolagents import OpenAIServerModel

# Determine which backend to use (from setup cell)
USE_GEMINI = os.environ.get("USE_GEMINI", "").lower() in ("1", "true", "yes")

if USE_GEMINI:
    # Use Gemini via OpenAI-compatible API
    model = OpenAIServerModel(
        model_id="gemini-2.0-flash-exp",
        api_base="https://generativelanguage.googleapis.com/v1beta/openai/",
        api_key=os.getenv("GOOGLE_API_KEY"),
    )
    print("⚙️ Using Gemini backend (gemini-2.0-flash-exp)")
else:
    # Use OpenAI (default)
    model = OpenAIServerModel(
        model_id="gpt-4o-mini",
        api_base="https://api.openai.com/v1",
        api_key=os.getenv("OPENAI_API_KEY"),
    )
    print("⚙️ Using OpenAI backend (gpt-4o-mini)")

print(model, dir(model))


## 2) Créer des outils (@tool)

In [ ]:
from smolagents import tool
import re

@tool
def calculator(expr: str) -> str:
    """
    Évalue une expression arithmétique simple.

    Args:
        expr (str): Expression arithmétique à évaluer. Autorisés: chiffres, espaces, point décimal,
                    et opérateurs +, -, *, / ainsi que les parenthèses ( et ).
    """
    if not re.fullmatch(r"[0-9+\-*/()\s\.]+", expr):
        return "Expression non autorisée."
    try:
        return str(eval(expr, {"__builtins__": {}}, {}))
    except Exception as e:
        return f"Erreur: {e}"

calculator("2+2")

In [ ]:
KB = {
  "jbv": "Jambon-beurre Vegan",
  "mtmgapd": "Mange ta main et garde l'autre pour demain",
}

@tool
def kb_lookup(term: str) -> str:
    """
    Retourne une petite définition depuis la KB locale (clé exacte).

    Args:
        term (str): clé exacte à récupérer dans la KB locale

    Returns:
        str: définition associé au term
    """
    return KB.get(term.lower(), f"(inconnue) clés={list(KB)})")

## 3) CodeAgent — planifier et agir en écrivant du *code*

CodeAgent privilégie l’exécution directe du modèle

In [ ]:
from smolagents import CodeAgent

agent_code = CodeAgent(tools=[calculator, kb_lookup], model=model, add_base_tools=False)
agent_code.run("Calcule 37*19, puis définis 'mtmgapd' en une phrase")

## 4) ToolCallingAgent — *tool calls* JSON natifs du modèle

ToolCallingAgent est un agent Tool-first

In [ ]:
from smolagents import ToolCallingAgent

agent_tool = ToolCallingAgent(tools=[calculator, kb_lookup], model=model, add_base_tools=False)
agent_tool.run("Calcule 37*19, puis définis 'mtmgapd' en une phrase")

## 5) Mini comparaison qualitative

In [ ]:
prompts = [
  "Calcule 23*51 et donne une phrase d'explication.",
  "Donne la définition de 'mtmgapd' et un exemple très court.",
]
for p in prompts:
    print("\n=== Prompt ===", p)
    print("\n--- CodeAgent ---")
    print(agent_code.run(p))
    print("\n--- ToolCallingAgent ---")
    print(agent_tool.run(p))

## 6) Questions
1. Différence conceptuelle entre **CodeAgent** et **ToolCallingAgent** ?  
2. Quand préféreriez-vous l’un ou l’autre ? (indices : sandbox, latence, traçabilité)  
3. Où placeriez-vous un **WebSearchTool** ou un **RAG** dans ce TP ?

In [ ]:
from smolagents import Tool
from duckduckgo_search import DDGS

class WebSearchTool(Tool):
    name = "web_search"
    description = "Recherche web via DuckDuckGo (résumés courts)."

    input_type = "object"
    output_type = "object"

    inputs = {
        "query": {
            "type": "string",
            "description": "Requête de recherche web"
        }
    }

    outputs = {
        "result": {
            "type": "string",
            "description": "Résumé des résultats"
        }
    }

    def forward(self, query: str):
        with DDGS() as ddgs:
            results = ddgs.text(query, max_results=5)

        snippets = [r.get("body", "") for r in results]
        return {"result": "\n".join(snippets[:3])}


In [ ]:
from smolagents import CodeAgent, WebSearchTool, InferenceClientModel

model = InferenceClientModel()
agent = CodeAgent(tools=[WebSearchTool()], model=model, stream_outputs=True)

agent.run("Fais une recherche web : Quel est l'âge de Danny Seagren ?")

In [ ]:
from smolagents import CodeAgent

search_tool = WebSearchTool()

agent = ToolCallingAgent(
    tools=[search_tool],
    model=model,
    add_base_tools=False
)

agent.run("Fais une recherche web : Quel est l'âge de Danny Seagren ?")


## 🔎 Bonus — Sandbox & WebSearchTool (optionnel)

In [ ]:
# Indices:
# - add_base_tools=True ajoute un interpréteur Python sandboxé & web search (DuckDuckGo) selon l'env.
# - Pour un vrai sandbox, voir la doc smolagents (E2B/Modal/Docker).
from smolagents import CodeAgent
agent_code2 = CodeAgent(tools=[kb_lookup], model=model, add_base_tools=True)
agent_code2.run("Quel est le produit de 91*97 puis fais une recherche web sommaire sur 'RAG' ?")

In [ ]:
import os
from smolagents import ToolCollection, CodeAgent, InferenceClientModel
from mcp import StdioServerParameters

model = InferenceClientModel()

server_parameters = StdioServerParameters(
    command="uvx",
    args=["--quiet", "pubmedmcp@0.1.3"],
    env={"UV_PYTHON": "3.12", **os.environ},
)

with ToolCollection.from_mcp(server_parameters, trust_remote_code=True) as tool_collection:
    agent = CodeAgent(tools=[*tool_collection.tools], add_base_tools=True, model=model)
    agent.run("Please find a remedy for hangover.")